# Cycle GAN 

## Pix2Pix

### Loss

$\sum_{(x,y)} ||y - G(x)||_{1} + L_{GAN} (G(x), y)$

> $G(x)$: generated output image (photo wanting to be seen as real)
>
> $y$: ground truth (real photo)

- GAN 의 일반적인 Loss $+$ L1 Loss (pixel to pixel difference)

    - L1 Loss 를 이용해서 기존 original photo $y$ 의 shape 를 그대로 유지

### 단점

- input 과 original photo 를 pair 로 사용해야함

> e.g.) 흑백 사진을 컬러사진으로 바꾸는 작업을 하고싶다..
>
> - Input 사진으로는 흑백사진 (input $x$)
>
> - Input 의 **컬러버전 사진 (ground truth $y$) 을 함께 가지고 있어야** training 가능 $\rightarrow$ Self **supervised learning**
>
> - 만약에 돈이 많아서, 사진사를 고용해서 갖고있는 흑백사진의 실제 장소에 가서 같은 사진을 컬러로 찍으면 좋겠지만.. $\rightarrow$ Out of budget
>
> $\Rightarrow$ 그 ground truth 를 구하기 어렵기 때문에, 그거 없이 흑백사진만 가지고 바로 변환해보자는 취지에서 *Cycle GAN* 탄생


## Cycle GAN

### 목표

- *pix2pix* 의 경우 training data 를 구성할때, 흑백사진과 함께 항상 정답이 되는 컬러사진을 함께 짝지어줘야함 (matching pair)

    **HOWEVER,**

- 그 정답이 되는 pair 가 항상 존재하는 것은 아님

- 우리가 그 Input 에 정확히 대응되는 컬러사진을 구할수는 없지만, 인터넷에서 많은 양의 컬러사진을 쉽게 구할수는 있음

$\Rightarrow$ Input dataset (흑백) 이 있고, ground truth dataset (컬러) 가 있을때, 컬러의 *style* 을 흑백에 적용시키도록 학습 (**image translation**)

### Loss

#### GAN Loss

$L_{GAN} (G(x), y)$

- GAN 의 Loss 를 그대로 적용

- $G$ 가 Input 을 어느 정도 수정해서 output $G(x)$ 를 generate 했을때, target (ground truth) dataset 의 member 처럼 보이게 하자

---

**Problem..**

만약에 $G$ 가 Input (그림) 의 형태 다 무시하고, Input 의 정보는 다 잊은채로 target (사진) 과 거의 똑같은 이미지만 생성한다면?

(e.g. 그림을 사진으로 바꿔주는 GAN 모델)

- $D$ 입장에서는 $G(x)$ 가 사진처럼 realistic 하게 보이기 때문에 그냥 True 로 분류해버림

    - 모델이 원치 않는 이상한 방향으로 학습하는것을 딱히 막을 방도가 없음
    

- 하지만 우리가 원하는 것은 그 그림의 모양을 그대로 유지하면서 photo-realistic 하게 보이도록 하는 것이지, 그냥 아무 사진이나 원하는 것이 아님


> **정리**
> 1. **Input 이미지를 무시**
>
> 2. 어떠한 Input 이 들어오든 간에 똑같은 Output 을 생성 (**Input 과 관련이 없는 label 생성**) $\rightarrow$ ***Mode Collapse***
>
> $\Rightarrow$ 의미없는 모델

---


#### Key-Concept

$G$ 가 Input 을 수정하기는 하는데, **다시 원래 그림이 복구 가능한 정도**까지만 변형해라

- $G$ 를 통해 Input 을 단순히 사진으로 바꾸는것 뿐만아니라,

- $F$ (new generator network) 를 통해 그 **사진을 다시 원래 Input 으로도 바꿀수 있어야**함


#### Cycle Loss

$L_{GAN}(G(x), y) + $ <span style="color: blue">$||F(G(x)) - x||_{1}$</span>

> $G(x)$ should just look photo-realistic and $F(G(x))$ should be $F(G(x)) = x$
>
> where $F$ is the **inverse deep network (역함수)** 

- $G$ (그림 $\rightarrow$ 사진) 과 $F$ (사진 $\rightarrow$ 그림) 을 동시에 훈련

- L1 Loss (pixel level) 을 사용한다는 점에서 pix2pix 와 비슷하지만, 그 대상을 network 로 직접 반대방향으로 $F(G(x))$ 돌려서 만들어냈다는 점이 다름


#### Cycle Loss (2)

${L_{GAN}(G(x), y) + ||F(G(x)) - x||_{1}} + {L_{GAN}(F(y), x) + ||G(F(y)) - y||_{1}}$

>- Generator 1 $G$
>
>    - $x$: 그림
>
>    - $G$: 그림 $\rightarrow$ 사진
>
>    - $G(x)$: $G$ 가 generate 한 **사진**
>
>
>- Generator 2 $F$
>
>    - $y$: 사진
>
>    - $F$: 사진 $\rightarrow$ 그림
>
>    - $F(y)$: $F$ 가 generate 한 **그림**


**1. 그림 $\rightarrow$ 사진 모델**

- $G$ 가 만들어내는 역할
    
- $F$ 가 되돌리는 역할
    
    
**inversely...**


**2. 사진 $\rightarrow$ 그림 모델**

- $F$ 가 만들어내는 역할
    
- $G$ 가 되돌리는 역할


<img src="./images/cycle_loss.png" style="width: 600px"></img>

> #### 결론
>
>$\rightarrow$ 반대로 바꾸면 똑같으니까 **둘다 같이하자!**
>
>($F$ 가 단순히 원래대로 되돌리는것 뿐만 아니라, 다른 사진을 그림으로 바꿀수 있는 general 한 능력 또한 가지게 되기 때문에 훨씬 학습을 잘하더라)

#### 참고

**$G$ 가 generate 한 input $G(x)$ 를 다시 Input 으로해서 학습시키는 모델은 어떨까?**

- 우선 training 시간이 많이 걸림

- $G(x)$ 자체가 깨끗한 Input 이 아닌, 생성된 corrupted input 이기 때문에, 막상 해보면 원하는 결과와는 동떨어진 (기존 Input 을 잘 반영하지 못한) 결과가 생성됨

    - corrupted input 은 gradient 가 자기 마음대로? 학습되도록 만듦
    
    
- 다만, 추후 Cycle GAN 의 성능이 좋아진다면, 이 방식을 이용했을때 훨씬 좋은 결과를 나타낼것이라고 생각함

---

**Cycle GAN 과 Auto-Encoder**

- Cycle GAN 은 Auto-Encoder 에서 latent vector (bottleneck) 를 사진 $G(x)$ 로 대체한 것이라고 볼수 있음

    - 그림 $\rightarrow$ 사진 ($\approx$latent vector) $\rightarrow$ 그림
    
---

**Disco GAN 과의 차의점?**

- Disco GAN 은 style 을 유지하면서 모양자체를 변경

- Cycle GAN 은 모양 (방향성) 을 그대로 유지하면서 style 을 변경

### Model Details (Generator $G$)

#### Encoder to Decoder

<img src="./images/encoder_decoder.png" style="width: 300px"></img>

- Disco GAN 에서 사용한 architecture

- encoder - decoder 사이에 bottleneck (latent vector) 를 만들어서, 중요한 특징을 압축했다가 다시 풀어내는 방식

- 중요한 특징들만을 뽑아냈기 때문에, 그것을 풀어낼때 급진적인 변화 (모양 변경 등) 에 적합

- 그러나 latent vector 는 기존의 정보를 저장하는데 한계가 존재

#### U-Net

<img src="./images/U_net.png" style="width: 300px"></img>

- Cycle GAN 에서 사용한 architecture

- **Encoder-decoder network $+$ skip-connection**

- skip-connection 기법을 사용하여 기존의 정보 (details) 를 뒤에 (last layer) 까지 전달

    - 많은 detail 을 그대로 가져갈수 있음
    
- 사진 $\rightarrow$ 그림 모델에서 **고해상도**를 유지하는데 필수적

- 그러나 skip-connection 을 사용함으로 인해 depth 를 깊게 가져갈수 없어서, 학습결과가 만족스럽지 못함

#### Fully connected ResNet

<img src="./images/ResNet.png" style="width: 350px"></img>
> ResNet block $\times$ 9

- 모델을 깊고 복잡하게 구성 가능

- skip-connection 기법이 사용되고 $+$ bottleneck 이 없음 $\rightarrow$ 기존 details 를 그대로 가져갈수 있음

    - 다만 메모리를 매우 많이 잡아먹음 $\rightarrow$ 학습할수있는 parameter 의 갯수가 제한적 $\rightarrow$ 큰 형태변화를 만들기에는 무리

### Example Results

#### Each combination of Loss terms

<img src="./images/term_combi_result.png" style="width: 700px"></img>
> Components (Terms)
>- L1 Loss
>- GAN Loss
>- forward L1 Loss
>- backward L1 Loss

1. Cycle alone : 기본 L1 Loss (pixel level difference)

2. GAN alone : 기존 GAN Loss ($L_{GAN}(G(x), y)$) 

3. GAN $+$ forward : GAN Loss $+$ forward L1 Loss (그림 $\rightarrow$ 사진)

3. GAN $+$ backward : GAN Loss $+$ backward L1 Loss (그림 $\leftarrow$ 사진)

4. Cycle GAN (ALL terms included) : ${L_{GAN}(G(x), y) + ||F(G(x)) - x||_{1}} + {L_{GAN}(F(y), x) + ||G(F(y)) - y||_{1}}$ 

> 당연히 모든 요소가 포함됬을때 나오는 결과가 가장 만족스러움

#### Reconstructed Images $F(G(x))$ for test image
<img src="./images/reconstructed.png" style="width: 500px"></img>

- $G(x)$ 가 뭐가 생성되었던 간에, 되돌리는 것을 정말 잘함

- $F$ 가 가능한한 원래대로 잘 되돌릴수 있어야 학습이 원하는 방향으로 진행가능 (constraints)

### Model Details (Objective function)

#### GANs with cross-entropy loss

$L_{GAN}(G, D_{Y}, X, Y) = E_{y \sim P_{data}(y)}[logD_{Y}(y)] + E_{x \sim P_{data}(x)}[log(1 - D_{Y}(G(x)))]$

> <img src="./images/cross_entropy_loss.png" style="width: 200px"></img>

- 기존 GAN Loss

- $x > 0$ 일경우, Loss 값이 거의 0 에 수렴하므로 gradient 가 거의 나타나지 않음 (***vanishing gradient***) $\rightarrow$ 학습 잘 안됨

---

#### Least square GANs

$L_{GAN}(G, D_{Y}, X, Y) = E_{y \sim P_{data}(y)}[(D_{Y}(y) - 1)^2] + E_{x \sim P_{data}(x)}[(D_{Y}(G(x)))^2]$

> <img src="./images/ls_loss.png" style="width: 200px"></img>

- Least Square 방식 활용

- Discriminator $D$ 가

    - $[(D_{Y}(y) - 1)^2]$ : 진짜인 $y$ 에는 가능한한 1 에 가까운 값
    
    - $[D_{Y}(G(x))^2]$ : 가짜인 $G(x)$ 에는 가능한한 0 에 가까운 값
    
    
- vanishing gradient 가 없으므로, 훈련이 훨씬 안정적으로 진행됨

> Stable training $\rightarrow$ No Mode collapse, No noisy results $\rightarrow$ Better result quality

### L1 Loss ($y$)

**$||F(G(y)) - y||_{1}$ is most important!**

- Cycle GAN 은 pix2pix 와 달리 Input 에 대응하는 정답이 주어지지 않음

- $F$ 를 통해 가상의 정답 (말 $\rightarrow$ <span style="color: blue">얼룩말</span> $\rightarrow$ 말) 을 주어줌으로써 훈련을 안정적으로 진행

#### New L1 Loss

<img src="./images/l1_loss_important.png" style="width: 400px"></img>

**3 Losses**

- GAN Loss : 말 $\rightarrow$ 얼룩말

- Reconstruction Loss : 얼룩말 $\rightarrow$ 말

- <span style="color: blue">Identitiy Loss</span> : 얼룩말 (Input $x$) $\rightarrow$ 얼룩말

---

- 이 Network 는 항상 얼룩말을 generate 함

- 그러면 얼룩말 사진 자체를 Input 으로 줬을때, 아무런 변화없이 그대로 나와야 함

> $x =$ 얼룩말 $\rightarrow$ **Identitiy function $G$** (항등식)  $\rightarrow$ $G(x) =$ 같은 얼룩말

#### L1 Identity Loss

<img src="./images/l1_identity_loss.png" style="width: 450px"></img>

- **일반 Cycle GAN** $\Rightarrow$ 수식적으로는 문제가 없지만, 뭔가 의도와는 다른 결과

    - 색상이 반전되는 경우
    
    - 분명 낮 사진을 넣었는데, 밤 분위기가 출력되는 경우
    

- **Cycle GAN $+$ Identity L1 Loss** ($L_{identity}$) $\Rightarrow$ 훨씬 안정적이고, 비슷한 결과

    - 모델 자체의 성능에 거의 영향을 주지 않음
    
    - 단, loss 가 추가됬기 때문에 formulation 이 복잡해지고, 학습시간이 더 오래걸림
    
    - 빼도 상관없기는 한데, **generate 할 이미지의 quality** 를 중요시한다면 넣을것
    
    

### Solving Training problem of GAN

#### Inconsistent training results

<img src="./images/inconsistent_training.png" style="width: 600px"></img>

- GAN 은 **Adverserial** 하기 때문에, 기본적으로 training 이 힘들다는 특징을 가지고 있음

    - $G$enerator 의 Loss 는 $D$iscriminator 로부터 나오고, $D$iscriminator 의 Loss 는 $G$enerator 로부터 나옴
    

- 위의 사진에서 <span style="color: blue">파란색</span> 선을 잘보면,

    - $D$ 가 첫번째 training 에서는 계속 real 로 분류 ($output > 0.5$)
    
    - 그런데 두번째 training 할때는 계속 fake 로 분류 ($output < 0.5$)
    
> **discriminator $D$ shows very different trajectories for each training procedure**

#### Solution 1: multiple $D$s

여러개의 discriminator 들을 학습시켜 그 결과를 평균 (ensemble)

$\rightarrow$ Network 를 추가로 더 학습시키는 것

$\rightarrow$ 컴퓨터 성능이 매우매우 좋아야함 ;;


#### Solution 2: replay buffer

***replay buffer*** : 

- reinforcement learning 에서 사용되는 기법으로 이전에 generator 가 생성한 이미지를 주기적으로 discriminator 에게 함께 보여줌

- discriminator 가 generator 의 예전 행동까지 함께 고려해서 결정을 내리도록함

$\Rightarrow$ Network 학습할때마다 다르게 행동하는 문제를 어느정도 해결

### 한계 (failure)

#### 모양을 바꾸기가 너무 힘듦

<img src="./images/change_shape.png" style="width: 500px"></img>

**Architecture Review**

- ***Encoder to Decoder*** : bottleneck (latent vector) 로 중요한 특징만을 뽑아내는것 $\rightarrow$ 형태 변형에 적합

- ***ResNet*** : skip-connection 기법을 이용하여 기존 Input 의 Detail 을 지키는 방식

> $\Rightarrow$ 기존 형태를 유지하려고 하다보니, 색깔 (style) 은 잘 바꾸는데, **형태를 바꾸는 임무에는 어색**함 (Orange??)

#### Training data distribution (형태)

- 말 위에 사람이 앉아있는 Input 을 주면 Cycle GAN 은 어떤 결과를 낼까?

<img src="./images/horse_with_person.png" style="width: 400px"></img>


- 말 부분만 조랑말 무늬로 바꿔서는 $D$ 를 속일수 없음

- 결국 사람 포함 전부다 조랑말 무늬로 바꿔버림

> $\rightarrow$ 데이터를 다변화하는 방식으로 후속연구 필요
>
> $!$ 사실 말 $+$ 조랑말 사진 대신에, **사람이 앉은** 말 $+$ **사람이 앉은** 조랑말 사진을 training data 로 사용하면 잘 구분하기는 함

#### Training data size

- 생각보다 많은 데이터가 필요하지는 않음

- 각 카테고리당 최소 500 개, 보통 1000 개, 많게는 2000 개

- ResNet 을 이용하기 때문에, 한 장이라도 각 부분을 나눠서 학습한다고 했을때, 한장 이상을 학습하는 효과를 가져옴

- 어차피 모델의 목적이 명확하기 때문에, 오히려 overfitting 을 추구해서, 더 좋은 결과물을 만들어내는 것도 괜찮음

    - 말 $\rightarrow$ 조랑말 모델은 정확히 그 task 만 잘하면 됨 (딱히 generalize 할 필요 없음)

### Applied Projects (Applications)

**Domain Adaptation Application**

1. CG2Real : GTA5 screen $\rightarrow$ real street view

2. 기존 game 화면 $\rightarrow$ **눈 덮인 겨울**의 game 화면

3. real-time image translation : generate 10 frames per second $\rightarrow$ need more computing powers for commercial use

### Get training data

1. [flickr](https://www.flickr.com/) 에서 tag 로 검색해서 scrap

2. data 의 특성을 잘 이해할것.

- e.g. 모네 그림의 경우에는 대부분 '프랑스' 풍경이 많음 $\rightarrow$ 프랑스 사진을 사용하면 모델 (그림 $\rightarrow$ 사진) 이 일을더 잘할수도 있음 

---